### Задача

1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

In [1]:
# импортируем все библиотеки
import json # расшифровка json
import requests # запросы
import pandas as pd
from pprint import pprint # красивый принт

#### Получаем данные

In [3]:
token = '<ваш_токен>' # вместо слов "ваш_токен" нужно вставить свой токен «Яндекс.Метрики»

headers = {'Authorization': 'OAuth ' + token} #собераем headers

In [4]:
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage', # метрика ym:s:visits визиты и ym:s:robotPercentage процент роботов
          'dimensions': 'ym:s:lastsignTrafficSource,ym:s:hour', # групировки ym:s:lastsignTrafficSource источники трафика
          'date1': '2020-09-01',                                 # по атрибуции последний значимый источник и часам ym:s:hour
          'date2': '2020-09-30',  # данные за сентябрь указано в Задание 9.7.1. в тестовом диалоге 
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}

In [5]:
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers) # запрос при помощи библиотеки requests
print (response.status_code) # проверяем статус, у мнения точно 200

200


#### Преобразования таблиц и типов данных

In [6]:
metrika_data = response.json() # преобразовываем json словарь из словарей

In [7]:
pprint(metrika_data) # просто проверка ,что всё нормально

{'contains_sensitive_data': False,
 'data': [{'dimensions': [{'icon_id': '2',
                           'icon_type': 'traffic-source',
                           'id': 'organic',
                           'name': 'Search engine traffic'},
                          {'id': '16', 'name': '16:00'}],
           'metrics': [251.0, 24.70119522]},
          {'dimensions': [{'icon_id': '2',
                           'icon_type': 'traffic-source',
                           'id': 'organic',
                           'name': 'Search engine traffic'},
                          {'id': '15', 'name': '15:00'}],
           'metrics': [240.0, 26.25]},
          {'dimensions': [{'icon_id': '2',
                           'icon_type': 'traffic-source',
                           'id': 'organic',
                           'name': 'Search engine traffic'},
                          {'id': '11', 'name': '11:00'}],
           'metrics': [228.0, 25.87719298]},
          {'dimensions': [{'icon_id': '2',
 

In [8]:
def getMetrikaDataInListOfDicts(metrika_data): # функция анпакинга структуры json в приемлемый для pandas вариант
    list_of_dicts = [] # список для добавления данных
    dimensions_list = metrika_data['query']['dimensions'] # список групировок 
    metrics_list = metrika_data['query']['metrics'] # список метрк
    for data_item in metrika_data['data']: # цыкл по основным данным 
        d = {} # временный словарь 
        for i,dimension in enumerate(data_item['dimensions']): # цыкл для групировок
            d[dimensions_list[i]] = dimension['name'] # добавление к временному словарю
        for i,metric in enumerate(data_item['metrics']): # цыкл для метрик
            d[metrics_list[i]] = metric # добавление к временному словарю
        list_of_dicts.append(d) #  добавление временного словаря в список
    return list_of_dicts

In [9]:
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data) # передаём в функцию структуре json

### Преобразование типов

In [10]:
for i in metrika_list_of_dicts: # цыкл преобразования типов
    i['ym:s:visits']=int(i['ym:s:visits']) # int

### Основная таблица

In [11]:
metrika_df = pd.DataFrame(metrika_list_of_dicts) # создание датафрейма 
metrika_df.columns=['Source','Hour','Visits','Robot%'] # преименование столбцов 
metrika_df['RobotCount'] = (metrika_df['Visits']/100)*metrika_df['Robot%'] # приблизительное число роботов
metrika_df['RobotCount'] = metrika_df['RobotCount'].apply(lambda x: int(x)) # цыкл преобразования типов
display(metrika_df.head(10))

,Source,Hour,Visits,Robot%,RobotCount
0,Search engine traffic,16:00,251,24.701195,62
1,Search engine traffic,15:00,240,26.250000,63
2,Search engine traffic,11:00,228,25.877193,58
3,Search engine traffic,13:00,227,28.193833,64
4,Search engine traffic,12:00,225,24.888889,56
5,Search engine traffic,14:00,225,25.777778,58
6,Search engine traffic,17:00,209,26.315789,54
7,Search engine traffic,10:00,186,19.354839,36
8,Search engine traffic,21:00,166,17.469880,29
9,Search engine traffic,19:00,161,23.602484,37


#### 1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

In [12]:
print('{} общее количество визитов'.format(metrika_df['Visits'].sum())) # общее количество визитов
print('{} общее приблизительное количество роботов'.format(metrika_df['RobotCount'].sum()) ) # общее приблизительное количество роботов(полученное из процента)
print('{} средний процент роботов'.format(metrika_df['Robot%'].mean())) # средний процент роботов

4315 общее количество визитов
932 общее приблизительное количество роботов
19.81789199738318 средний процент роботов


Процент роботов в среднем, по потокам почти достиг 1/5 всего трафика сайта. Это не совсем плохой показатель, но требуется улучшения и установка более современного "ПО" по блокировке сеансов с роботами и подозрительными запросами.

#### 2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

#### Пропуски: обработаны при помощи "fill_value=0" - замена всех пропусков на ноль

In [13]:
# сводная таблица
metrika_df.pivot_table(values=['Robot%','Visits'], index='Source',aggfunc='mean',fill_value=0) # "fill_value=0" - замена всех пропусков на ноль

,Robot%,Visits
Source,,
Cached page traffic,10.000000,1.200000
Direct traffic,23.835393,31.541667
Internal traffic,25.000000,1.250000
Link traffic,27.249606,5.727273
Recommendation system traffic,0.000000,1.000000
Search engine traffic,18.963520,138.916667
Social network traffic,11.084055,3.636364


Лидеры по трафику ботов: прямые ссылки(Link traffic) и Internal traffic за ними переходы по рекламе(Direct) также трафик с поисковых систем(Search engine).

In [14]:
# сводная таблица
metrika_df.pivot_table(values='Visits', index='Source',aggfunc='sum',fill_value=0) # "fill_value=0" - замена всех пропусков на ноль

,Visits
Source,
Cached page traffic,6
Direct traffic,757
Internal traffic,10
Link traffic,126
Recommendation system traffic,2
Search engine traffic,3334
Social network traffic,80


Берём 3 самых репрезентативных источника: Direct traffic, Link traffic, Search engine traffic.

Search engine traffic повышение активности ботов явно просматривается с 80:00 до 19:00. С пиками в 13:00 и 17:00. и всплеском в 22:00

In [15]:
# сводная таблица Search engine traffic 
metrika_df.pivot_table(index='Hour',columns='Source',values=['Robot%','Visits'],fill_value=0)['Robot%'][['Search engine traffic']]

Source,Search engine traffic
Hour,
00:00,16.666667
01:00,15.094340
02:00,12.195122
03:00,10.714286
04:00,13.043478
05:00,4.000000
06:00,8.823529
07:00,9.836066
08:00,22.656250


Link traffic довольно мало переходов вообще, но из них рекордные 27% не живого трафика. Данные не постоянны и можно заключить, что ссылки используют чаше как раз для парсинга. Стоит установить отдельный фильтр для подобных запросов на сервер сайта.

In [16]:
# сводная таблица Link traffic
metrika_df.pivot_table(index='Hour',columns='Source',values=['Robot%','Visits'],fill_value=0)['Robot%'][['Link traffic']]

Source,Link traffic
Hour,
00:00,0.000000
01:00,0.000000
02:00,0.000000
03:00,100.000000
04:00,0.000000
05:00,0.000000
06:00,50.000000
07:00,33.333333
08:00,25.000000


Direct traffic данные достаточно не стабильны, есть пик в 13:00 в 40.6% что много, и весомо вредит, и более не выраженные, но всё еще высокие в 04:00, 09:00, 23:00.

In [17]:
# сводная таблица Direct traffic
metrika_df.pivot_table(index='Hour',columns='Source',values=['Robot%','Visits'],fill_value=0)['Robot%'][['Direct traffic']]

Source,Direct traffic
Hour,
00:00,20.000000
01:00,25.000000
02:00,18.181818
03:00,22.222222
04:00,33.333333
05:00,20.000000
06:00,27.272727
07:00,26.315789
08:00,23.809524


#### Вывод 

Трафик ботов не стабилен и зависит от источника. Сервер слабо прерывает такого рода трафик в случаях когда он переваливает на пиках почти за треть всех запросов, "ПО" сервера рекомендованною оснастить защитой, и массовыми проверками при помощи капчи.